In [0]:
%sql
CREATE TABLE tabledelta
(
  id INT,
  name STRING,
  salary INT
)
USING DELTA
LOCATION '/FileStore/deltasource/Source1'

In [0]:
%sql
ALTER TABLE tabledelta SET TBLPROPERTIES ('delta.enableDeletionVectors' = False)

In [0]:
%sql
insert into tabledelta
values
  (1,'Priya',10000),
  (2,'Rahul',20000),
  (3,'Shanti',30000)

In [0]:
%sql
select * from tabledelta

In [0]:
df = spark.readStream.table('tabledelta')

In [0]:
df.writeStream.format('delta') \
  .option('checkpointLocation','/FileStore/deltasource/sink1/checkpoint') \
  .option('path',"/FileStore/deltasource/sink1/data") \
  .trigger(processingTime = "10 seconds") \
  .start()

In [0]:
%sql
select * from delta.`/FileStore/deltasource/sink1/data`

In [0]:
%sql
DESCRIBE HISTORY tabledelta

In [0]:
df_new = spark.readStream.option('startingVersion',16).option('ignoreChanges','true').table('tabledelta')

df_new.writeStream.format('delta') \
  .option('checkpointLocation','/FileStore/deltasource/sink1/checkpoint_new') \
  .option('path',"/FileStore/deltasource/sink1/data_new") \
  .trigger(processingTime = "1 seconds") \
  .start()

In [0]:
%sql
select * from delta.`/FileStore/deltasource/sink1/data_new`

In [0]:
%sql
update tabledelta
set name = 'iron'
where id = 2